In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import random


### 1. Read in IBC data and separate to agreed data and not-doctor-notes data

In [30]:
#test_model = pd.read_csv('ibc_trainingdata_040219.csv')  # this data is taken from ibc data, there are 2000 charts set aside not being used in this dataset

In [3]:
test_model = pd.read_csv('/home/jovyan/work/Model_Training_Data/IBC_data_260319_1.csv')  # this is the more complete IBC data

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
test_model.head()

,Unnamed: 0,chart_id,code_id,text,start_offset,end_offset,deleted_reason,deleted,label,15char,25char,50char,75char,10word_after,10word_before,50word_after,50word_before,flag
0,0,002803945_161667176A_2016_001,E6601,e66.01,14251,14257,NaN,NaN,agreed,cess calories (E66.01).,due to excess calories (E66.01). ...,sessment Morbid obesity due to excess calori...,.7). Assessment Morbid obe...,"[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Fibromyalgia', '(M79.7).', 'Assessment', 'M...","[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Assessment/Plan', '#', 'Detail', 'Type', 'D...",0
1,1,002803945_161667176A_2016_001,E6601,e66.01,22445,22451,NaN,NaN,agreed,cess calories (E66.01). Impre,due to excess calories (E66.01). Impre...,sessment Morbid obesity due to excess calori...,ise. Assessment Morbid obe...,"[[').', 'Impression', 'encouraged', 'to', 'die...","[['-diet', 'exercise.', 'Assessment', 'Morbid'...","[[').', 'Impression', 'encouraged', 'to', 'die...","[['Impression', 'seeing', 'pain', 'management,...",0
2,2,002803945_161667176A_2016_001,E789,e78.9,14501,14506,NaN,NaN,agreed,m cholesterol (E78.9).,vated serum cholesterol (E78.9). ...,Assessment Elevated serum cholester...,izures (F44.5). Assessment ...,"[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['sequela', '(S06.9X9S).', 'Assessment', 'Pse...","[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['chronic', 'depression--multitude', 'sx', 's...",0
3,3,002803945_161667176A_2016_001,F329,f32.9,21557,21562,NaN,NaN,agreed,"sion, chronic (F32.9). Impression","jor depression, chronic (F32.9). Impressions...","ad injury. AssessmentMajor depression, chron...","etter w meds, flared w head injury. Assessme...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['w', 'meds,', 'flared', 'w', 'head', 'injury...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['accupuncturist,pain', 'management.', 'Asses...",0
4,4,002803945_161667176A_2016_001,F329,Major depression,17574,17590,NaN,NaN,agreed,"pap finding Major depression,08/10/2016 ch","abnormal pap finding Major depression,08/1...",:00 PM 3/36 unspecified abnormal pap findin...,07/28/1977 09/19/2016 06:00 PM 3/36 unspeci...,"[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['I.', '07/28/1977', '09/19/2016', '06:00', '...","[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['on', '05/04/2014.', 'Hallucination', '08/10...",0


#### a. Select not-doctor notes deletion reason data and chart list

In [7]:
deleteddata=test_model[test_model['deleted_reason']=='not_doctors_note']
print('length of deleted data is ')
print(len(deleteddata))
len_del = len(deleteddata)
not_doctors_note_charts = list(deleteddata['chart_id'])
print(not_doctors_note_charts)

length of deleted data is 
35074
['002803945_161667176A_2016_001', '002804844_199286931A_2016_001', '002805282_174608825B_2016_001', '002805282_174608825B_2016_001', '002805282_211548626A_2016_001', '002805282_211548626A_2016_001', '002805282_211548626A_2016_001', '002805282_211548626A_2016_001', '002805282_211548626A_2016_001', '002805282_211548626A_2016_001', '002806405_202308159A_2016_001', '002806405_202308159A_2016_001', '002806751_192204772A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '002807434_160326491A_2016_001', '00280743

In [10]:
len(not_doctors_note_charts)  # total instances related to it

35074

In [9]:
deleteddata['chart_id'].nunique()   # there are 8772 unique chart id in not-doctor-notes deleted data, not 33486

8772

#### b. Select agreed+added data, and chart list selection

In [26]:
# select agreed data, since it select flag==0, it includes the added case too
agreementdata=test_model[test_model['flag']==0]
print('length of agreement data is ')
print(len(agreementdata))

length of agreement data is 
173651


In [13]:
agreementdata['chart_id'].nunique()  # 30227 unique chart id for agreement data

30227

In [27]:
agreed_charts=list(agreementdata['chart_id'])

#### c. find the overlap charts, then exclude them

In [16]:
len(set(not_doctors_note_charts))

8772

In [28]:
len(set(agreed_charts))

30227

In [15]:
overlaplist=list(set(not_doctors_note_charts) & set(agreed_charts))
len(overlaplist)  

5517

In [21]:
exclusive_notdoctornote_chart=list(set(not_doctors_note_charts)-set(overlaplist))  # 8772-5517
print(len(exclusive_notdoctornote_chart))
print(len(set(exclusive_notdoctornote_chart)))  # after removing overlap charts, only 3255 chart ids for not-doctor-notes

3255
3255


In [29]:
exclusive_agreed_charts=list(set(agreed_charts)-set(overlaplist))
print(len(exclusive_agreed_charts))        # 30227-5517

24710


#### d. Select  1000 training chart-id from exclusive_not doctor notes chart list, and set aside 1000 for testing

In [30]:
# randomly select 1000 charts, but it should be done after obtaining exclusive list, that has removed the overlap list
selno=1000
# randomly select 1000 as training
notdoctorlist1000training=random.sample(range(len(exclusive_notdoctornote_chart)), selno) 
# training 1000 chart_id list
selectnotdoctornotes_train1000=list(exclusive_notdoctornote_chart[i] for i in notdoctorlist1000training)
# save to file
df = pd.DataFrame(data={"chart_id": selectnotdoctornotes_train1000 })
df.to_csv("ndn_train1000_chartlist.csv", sep=',',index=False)

print(len(selectnotdoctornotes_train1000))


1000


In [31]:
# remove this 1000 charts from list
rest_ndn_list= list(set(exclusive_notdoctornote_chart)-set(selectnotdoctornotes_train1000))
# randomly select 1000 as testing
notdoctorlist1000testing=random.sample(range(len(rest_ndn_list)), selno) 
# testing 1000 chart_id list
selectnotdoctornotes_test1000=list(rest_ndn_list[i] for i in notdoctorlist1000testing)
# save to file
df = pd.DataFrame(data={"chart_id": selectnotdoctornotes_test1000 })
df.to_csv("ndn_test1000_chartlist.csv", sep=',',index=False)
print(len(selectnotdoctornotes_test1000))

1000


In [32]:
# sanity check, no overlap, then all good
len(list(set(selectnotdoctornotes_train1000) & set(selectnotdoctornotes_test1000)))

0

In [33]:
len(list(set(selectnotdoctornotes_train1000) & set(overlaplist)))

0

In [34]:
len(list(set(selectnotdoctornotes_test1000) & set(overlaplist)))

0

#### e. Select 1000 training chart-id from exclusive_agreed chart list, and set aside 1000 for testing

In [36]:
selno=1000
# randomly select 1000 as training
agreedlist1000training=random.sample(range(len(exclusive_agreed_charts)), selno) 
# training 1000 chart_id list
selectagreed_train1000=list(exclusive_agreed_charts[i] for i in agreedlist1000training)
# save to file
df = pd.DataFrame(data={"chart_id": selectagreed_train1000 })
df.to_csv("agreed_train1000_chartlist.csv", sep=',',index=False)

print(len(selectagreed_train1000))

1000


In [37]:
# remove this 1000 charts from list
rest_agreed_list= list(set(exclusive_agreed_chart)-set(selectagreed_train1000))
# randomly select 1000 as testing
agreedlist1000testing=random.sample(range(len(rest_agreed_list)), selno) 
# testing 1000 chart_id list
selectagreed_test1000=list(rest_agreed_list[i] for i in agreedlist1000testing)

# save to file
df = pd.DataFrame(data={"chart_id": selectagreed_test1000 })
df.to_csv("agreed_test1000_chartlist.csv", sep=',',index=False)

print(len(selectagreed_train1000))

1000


In [ ]:
# sanity check, no overlap or 0 is good

In [38]:
len(list(set(selectagreed_train1000) & set(selectagreed_test1000))) # no overlap between training and testing data

0

In [39]:
len(list(set(selectagreed_train1000) & set(overlaplist)))  # no overlap with overlap list

0

In [40]:
len(list(set(selectagreed_test1000) & set(overlaplist)))

0

In [41]:
len(list(set(selectagreed_test1000) & set(selectnotdoctornotes_test1000)))

0

In [42]:
len(list(set(selectagreed_train1000) & set(selectnotdoctornotes_train1000)))

0

In [57]:
(selectagreed_test1000).sort()

In [58]:
selectagreed_test1000

['002812424_171265152A_2016_001',
 '002816884_172241560A_2016_001',
 '002821919_278386922A_2016_001',
 '002884091_196202246A_2016_002',
 '002884091_197301226A_2016_001',
 '002884607_195363627A_2016_001',
 '003189221_128282004A_2016_002',
 '003189221_192260291A_2016_001',
 '003189221_199344819A_2016_002',
 '003290490_173340463A_2016_001',
 '003290490_209242811D_2016_001',
 'IBXRMA2018_MULT_1163307070010001_HMK_197383215A_JEFFREY_WENTZ_003102386_2017',
 'IBXRMA2018_MULT_1163423250010001_HMK_169328156A_IRENE_SERPENTE_002804905_2017',
 'IBXRMA2018_MULT_1163552550010001_HMK_181287363A_THOMAS_RICHARDSON_002805068_2017_20180502150249',
 'IBXRMA2018_MULT_1163579850010001_HMK_188561556A_HWA_YU_002812337_2017',
 'IBXRMA2018_MULT_1163663570010001_HMK_179341552A_ANNA_CANTWELL_002811550_2017_20180517112028',
 'IBXRMA2018_MULT_1163716830010001_HMK_164428662A_MARIA_FAMA_002820440_2017',
 'IBXRMA2018_MULT_1163915960010001_HMK_171204543B_DOROTHY_CHRISMAN_002813095_2017',
 'IBXRMA2018_MULT_1163924380010